In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv
/kaggle/input/payload-resume2/Aninditaa_Resume_ML.pdf


In [96]:
!pip install PyPDF2

In [97]:
# -*- coding: utf-8 -*-
# Modified for Kaggle: Batch Job Recommendations & Total Experience Calculation

# Install necessary packages
import pandas as pd
import numpy as np
import re
import spacy
import os
from datetime import datetime
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")



In [98]:
# Check if the pickle file exists
if os.path.exists("job_df.pkl"):
    job_df = pd.read_pickle("job_df.pkl")
    print("DataFrame loaded from pickle file.")
else:
    # Load the dataset from Kaggle's "Input" section
  #  csv_file_path = "/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv"  # Replace with your dataset path
    job_df = pd.read_csv("/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv")  # Replace with your dataset path
    job_df.to_pickle("job_df.pkl")
    print("DataFrame saved successfully as job_df.pkl.")

 
# Step 1: Load the Resume

def load_resume(file_path):
    if file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = "".join([page.extract_text() for page in reader.pages ])
    elif file_path.endswith('.docx'):
        import docx2txt
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text


DataFrame loaded from pickle file.


In [99]:
# # Step 2: Extract Information and Fix Total Experience
# def extract_resume_info(resume_text):
#     skills = {
#         'Languages': [],
#         'Tools': [],
#         'Frameworks/Technologies': []
#     }
#     education = []
#     experience = []
#     total_experience_months = 0

#     # Extract Skills Section
#     skills_pattern = re.search(r"SKILLS:([\s\S]*?)(?=(EDUCATION|PROJECTS|EXPERIENCE|$))", resume_text, re.IGNORECASE)
#     if skills_pattern:
#         skills_text = skills_pattern.group(1).split('\n')
#         for line in skills_text:
#             if 'Languages:' in line:
#                 skills['Languages'] = [item.strip() for item in line.split(':')[1].split(',')]
#             elif 'Tools:' in line:
#                 skills['Tools'] = [item.strip() for item in line.split(':')[1].split(',')]
#             elif 'Frameworks/Technologies:' in line:
#                 skills['Frameworks/Technologies'] = [item.strip() for item in line.split(':')[1].split(',')]

#     # Extract Education Section
#     edu_pattern = re.findall(r"(Master|Bachelor).*?in ([^,]+) from ([^,]+), (\w+ \d{4}) - (\w+ \d{4}|Present)", resume_text, re.IGNORECASE)
#     for degree, field, institution, start, end in edu_pattern:
#         education.append(f"{degree} of {field}, {institution}, {start} - {end}")

#     # Extract Work Experience Section
#     exp_pattern = re.findall(r"(.*?) at (.*?), (.*?)\s*-\s*(\w+ \d{4}|Present)", resume_text, re.IGNORECASE)
#     for role, company, location, dates in exp_pattern:
#         experience.append(f"{role}, {company}, {location}, {dates}")
#         date_matches = re.findall(r"(\w+ \d{4})", dates)
#         if len(date_matches) == 2:
#             try:
#                 start_date = datetime.strptime(date_matches[0], "%b %Y")
#                 end_date = datetime.strptime(date_matches[1], "%b %Y") if date_matches[1] != 'Present' else datetime.now()
#                 months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
#                 total_experience_months += max(months, 0)
#             except ValueError:
#                 print(f"DEBUG: Error parsing dates {dates}")

#     # Convert months to years
#     total_experience_years = round(total_experience_months / 12, 1)

#     return {
#         'skills': skills,
#         'education': education,
#         'experience': experience,
#         'total_experience_years': total_experience_years
#     }

# Extract information from the resume using OpenAI GPT
!pip install openai
import openai

openai.api_key = 'sk-svcacct-CHBAA1llvXYtby7INMBGJbqPQ5SSWvqiHHqnAOAWqNmUmmS6oVJ_eX0Ixab8wXFmKwT3BlbkFJupGRsdBRq-ZkCUdnD42l9bO_n3lHhLi-QvFKSPHAAUge4RQt7GBVUKcx4uXh8tKzIA'


def extract_information_from_resume(resume_text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts structured information from resumes."},
            {"role": "user", "content": f"Extract the following information from the resume below:\n\n1. Skills\n2. Education or Qualifications\n3. Work Experience (including roles and companies)\n4. Total Years of Experience\n\nResume:\n{resume_text}"}
        ],
        max_tokens=1000,
        temperature=0.5
    )

    #extracted_info = response['choices'][0]['message']['content'].strip()
    extracted_info = response.choices[0].message.content
    return extracted_info

extracted_info = extract_information_from_resume(resume_text)
print("Extracted Information from Resume:\n", extracted_info)

Extracted Information from Resume:
 1. Skills:
- Languages: Python, Java, Golang, C/C++
- Tools: GCP, Vertex AI, Kubernetes, Helm, Docker, git
- Frameworks/Technologies: LangChain, Mediapipe, TensorFlow, OpenCV, scikit-learn, Redis, MongoDB

2. Education:
- Master of Science in Computer Science (Thesis) from Virginia Tech, Aug 2024 - Present
- Bachelor of Technology in Computer Science and Engineering from Manipal University Jaipur, July 2019 - June 2023

3. Work Experience:
- Software Engineer at Harness [Software Delivery Platform], Bangalore, India
  - Aug 2023 - July 2024
  - Software Engineering Intern
    - Feb 2023 - Aug 2023
- Research Intern at CVIT Lab - IIIT Hyderabad, Hyd, India
  - June 2022 – Sept 2022

4. Total Years of Experience: 
- Based on the provided information, Aninditaa Chauhan has approximately 2 years of work experience.


In [100]:
# Step 3: Display Extracted Information
# def display_extracted_info(info):
#     print("\nExtracted Information from Resume:")
#     print("\n1. Skills:")
#     for skill in info['skills']:
#         print(f"- {skill}")
#     print("\n2. Education:")
#     for edu in info['education']:
#         print(f"- {edu}")
#     print("\n3. Work Experience:")
#     for exp in info['experience']:
#         print(f"- {exp}")
#     print(f"\n4. Total Years of Experience: {info['total_experience_years']} years")

# Parse extracted information
def parse_extracted_info(extracted_text):
    skills_match = re.search(r'Skills:(.*?)(Education|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    education_match = re.search(r'Education:(.*?)(Experience|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    experience_match = re.search(r'Experience:(.*?)(Total Years of Experience|$)', extracted_text, re.DOTALL | re.IGNORECASE)
    years_match = re.search(r'Total Years of Experience:(.*)', extracted_text, re.DOTALL | re.IGNORECASE)

    skills = [skill.strip() for skill in skills_match.group(1).split(',')] if skills_match else []
    education = [degree.strip() for degree in education_match.group(1).split(',')] if education_match else []
    experience = [exp.strip() for exp in experience_match.group(1).split(',')] if experience_match else []
    total_years = years_match.group(1).strip() if years_match else "0"

    return skills, education, experience, total_years

skills, education, experience, total_years = parse_extracted_info(extracted_info)


In [101]:
# Step 4: Vectorization and Similarity Calculation (Batch PRocessing)

# def calculate_similarity(resume_text, job_descriptions):
#     vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
#     combined_texts = [resume_text] + job_descriptions
#     tfidf_matrix = vectorizer.fit_transform(combined_texts)
#     similarity_matrix = cosine_similarity(tfidf_matrix)
#     return similarity_matrix[0, 1:]


def calculate_similarity_in_batches(resume_text, job_descriptions, batch_size=5000):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    similarities = []

    for i in range(0, len(job_descriptions), batch_size):
        batch = job_descriptions[i:i + batch_size]
        combined_texts = [resume_text] + batch
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
        batch_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
        similarities.extend(batch_similarities)

    return similarities

In [102]:
# Step 5: Match and Rank Jobs

# def match_jobs(resume_text, job_db):
#     job_db_subset = job_db.copy()
#     similarities = calculate_similarity(resume_text, job_db_subset['description'].tolist())
#     job_db_subset['similarity_score'] = similarities  # No normalization
#     sorted_jobs = job_db_subset.sort_values(by='similarity_score', ascending=False)
#     return sorted_jobs

def match_jobs(resume_text, job_db_path, chunksize=5000):
    top_matches = []
    for chunk in pd.read_csv(job_db_path, chunksize=chunksize):
        similarities = calculate_similarity_in_batches(resume_text, chunk['description'].tolist())
        chunk['similarity_score'] = similarities
        top_matches.append(chunk.sort_values(by='similarity_score', ascending=False).head(10))
    return pd.concat(top_matches).sort_values(by='similarity_score', ascending=False).head(10)

In [105]:

# Main Execution
if __name__ == "__main__":
    # Load the resume
    resume_file_path = "/kaggle/input/payload-resume2/Aninditaa_Resume_ML.pdf"  # Replace with your Kaggle path
    if os.path.exists(resume_file_path):
        resume_text = load_resume(resume_file_path)

        # Extract information from resume
        resume_info = extract_information_from_resume(resume_text)
        # display_extracted_info(resume_info)
        print(extracted_info)

        # Load job dataset
        job_dataset_path = "/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv"  # Replace with your dataset path
        if os.path.exists(job_dataset_path):
            job_df = pd.read_csv(job_dataset_path)

            # Match jobs and get recommendations
            recommended_jobs = match_jobs(resume_text, job_dataset_path)

            print("\nTop 10 Recommended Jobs:")
            print(recommended_jobs[['title', 'description', 'published_date', 'country', 'similarity_score']].head(10))
        else:
            print(f"Error: Job dataset file '{job_dataset_path}' not found.")
    else:
        print(f"Error: Resume file '{resume_file_path}' not found.")


1. Skills:
- Languages: Python, Java, Golang, C/C++
- Tools: GCP, Vertex AI, Kubernetes, Helm, Docker, git
- Frameworks/Technologies: LangChain, Mediapipe, TensorFlow, OpenCV, scikit-learn, Redis, MongoDB

2. Education:
- Master of Science in Computer Science (Thesis) from Virginia Tech, Aug 2024 - Present
- Bachelor of Technology in Computer Science and Engineering from Manipal University Jaipur, July 2019 - June 2023

3. Work Experience:
- Software Engineer at Harness [Software Delivery Platform], Bangalore, India
  - Aug 2023 - July 2024
  - Software Engineering Intern
    - Feb 2023 - Aug 2023
- Research Intern at CVIT Lab - IIIT Hyderabad, Hyd, India
  - June 2022 – Sept 2022

4. Total Years of Experience: 
- Based on the provided information, Aninditaa Chauhan has approximately 2 years of work experience.

Top 10 Recommended Jobs:
                                                   title  \
22612             1715 - writing an article on economics   
7294     ML/AI for Orthognathic